In [1]:
# rag_with_my_model.py
import torch
import fitz  # PyMuPDF
import re
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer, CrossEncoder
import sentencepiece as spm
from decoder_only_gpt import My_GPT_model

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Load tokenizer & model (tera trained model)
sp = spm.SentencePieceProcessor()
sp.load("hindi_tokenizer_new.model")

True

In [3]:
model = My_GPT_model(
    vocab_size=sp.get_piece_size(),
    num_layers=12,
    d_model=512,
    d_ff=2048,
    num_heads=8,
    seq_len=512
).to(DEVICE)

# Load final SFT checkpoint
model.load_state_dict(torch.load("full_sft_final.pt", map_location=DEVICE))
model.eval()

My_GPT_model(
  (decoder): Decoder(
    (embedding): Embedding(32768, 512)
    (layers): ModuleList(
      (0-11): 12 x Decoder_GPT_Block(
        (swi_glu): SwiGLU_FFN(
          (w1): Linear(in_features=512, out_features=1536, bias=False)
          (w2): Linear(in_features=512, out_features=1536, bias=False)
          (w3): Linear(in_features=1536, out_features=512, bias=False)
          (act): SiLU()
        )
        (masked_mha): Masked_MHA(
          (Q): Linear(in_features=512, out_features=512, bias=True)
          (K): Linear(in_features=512, out_features=512, bias=True)
          (V): Linear(in_features=512, out_features=512, bias=True)
          (fc_out): Linear(in_features=512, out_features=512, bias=True)
        )
        (rms_norm0): RMSNorm()
        (rms_norm1): RMSNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (norm): RMSNorm()
  )
  (lm_head): Linear(in_features=512, out_features=32768, bias=False)
)

In [4]:
# 2. Load embedding & reranker
embed_model = SentenceTransformer("intfloat/multilingual-e5-base")
reranker = CrossEncoder("cross-encoder/mmarco-mMiniLMv2-L12-H384-v1")

In [5]:
print("Vocab size from tokenizer:", sp.get_piece_size())
print("Model vocab size:", model.lm_head.out_features)  # should match

Vocab size from tokenizer: 32768
Model vocab size: 32768


In [26]:
def clean_hindi_text(text):
    if not text:
        return ""

    # Remove non-printable characters
    text = re.sub(r'[\x00-\x1F\x7F]', ' ', text)

    # Fix common PDF junk chars
    text = re.sub(r'[�•ﬁﬂ–—]', ' ', text)

    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [27]:
def fix_pdf_text(raw_text):
    text = re.sub(r'\s+', ' ', raw_text)
    text = re.sub(r'([ऀ-ॿ])([A-Za-z0-9])', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z0-9])([ऀ-ॿ])', r'\1 \2', text)
    return text.strip()

In [28]:
pdf_path = "story.pdf"
doc = fitz.open(pdf_path)

raw_text = ""
for page in doc:
    raw_text += page.get_text("text")

doc.close()

fixed_text = fix_pdf_text(raw_text)
cleaned_text = clean_hindi_text(fixed_text)

print(f"Fixed + Cleaned text snippet:\n{cleaned_text[:500]}")

Fixed + Cleaned text snippet:
सपनों का एक छोटा-सा कारवाँ सुबह की पहली किरण जब खिड़की से झाँकती है, तो लगता है जैसे कोई पुरानी दोस्त मुस्कुरा रही हो। वही दोस्त जो बचपन में कहती थी "बड़ा होकर क्या बनोगे?" मैंने जवाब दिया था "सब कुछ!" आज भी वही जवाब मेरे सीने में धड़कता है। सिर्फ अब थोड़ा संशय भी साथ चलता है, जैसे कोई पुराना जूता जो पैर में फिट तो है, पर थोड़ा काटता भी है। जीवन एक लंबी ट्रेन यात्रा है। कुछ स्टेशन पर चाय की केतली गरम रहती है, कुछ पर सिर्फ ठंडी हवा और खामोशी। फिर भी हर स्टेशन पर उतरने वाले लोग अलग-अलग कहानियाँ छो


In [29]:
cleaned_text

'सपनों का एक छोटा-सा कारवाँ सुबह की पहली किरण जब खिड़की से झाँकती है, तो लगता है जैसे कोई पुरानी दोस्त मुस्कुरा रही हो। वही दोस्त जो बचपन में कहती थी "बड़ा होकर क्या बनोगे?" मैंने जवाब दिया था "सब कुछ!" आज भी वही जवाब मेरे सीने में धड़कता है। सिर्फ अब थोड़ा संशय भी साथ चलता है, जैसे कोई पुराना जूता जो पैर में फिट तो है, पर थोड़ा काटता भी है। जीवन एक लंबी ट्रेन यात्रा है। कुछ स्टेशन पर चाय की केतली गरम रहती है, कुछ पर सिर्फ ठंडी हवा और खामोशी। फिर भी हर स्टेशन पर उतरने वाले लोग अलग-अलग कहानियाँ छोड़ जाते हैं। कभी कोई बूढ़ी दादी अपनी पोती को समझाती दिखती है "बेटा, धैर्य रखो, सब ठीक हो जाएगा।" कभी कोई नौजवान फोन पर चिल्लाता है "मैंने कहा ना, अगले महीने पैसा भेज दूँगा!" और मैं? मैं बस खिड़की से बाहर देखता हूँ। खेतों में हल चलाते बैल, दूर पहाड़ों पर बादल का खेल, और बीच-बीच में उड़ते पंछी जो शायद किसी सपने की ओर जा रहे हैं। कभी-कभी सोचता हूँ सपने सच होते हैं या हम सपनों को सच मान लेते हैं? शायद दोनों। क्योंकि जो इंसान सपना देखना छोड़ देता है, वह जीना भी धीरे-धीरे छोड़ देता है। कल एक बच्चे ने

In [30]:
#Chunking function (sentence level)
def chunk_text(text, max_tokens=250, overlap=50):
    tokens = sp.encode(text)
    chunks = []
    i = 0
    while i < len(tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(sp.decode(chunk))
        i += max_tokens - overlap
    return chunks

In [31]:
def sentence_chunk_text(text, max_chars=800, overlap_sentences=1):
    sentences = re.split(r'(?<=[।!?])\s+', text)

    chunks = []
    current_sents = []

    for sent in sentences:
        current_sents.append(sent)

        chunk_text = " ".join(current_sents)

        if len(chunk_text) >= max_chars:
            chunks.append(chunk_text.strip())

            # overlap: last N full sentences
            current_sents = current_sents[-overlap_sentences:]

    if current_sents:
        chunks.append(" ".join(current_sents).strip())

    return chunks

In [32]:
#Build FAISS index
def build_index(chunks):
    texts = ["passage: " + chunk for chunk in chunks]
    embeddings = embed_model.encode(texts, normalize_embeddings=True, batch_size=32)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings.astype(np.float32))
    return index, chunks

In [42]:
@torch.no_grad()
def generate(
    model,
    input_ids,
    max_new_tokens=120,
    temperature=0.75,
    top_p=0.92,
    repetition_penalty=1.08,
    top_k=40
):
    model.eval()
    generated = input_ids.clone()

    for _ in range(max_new_tokens):
        outputs = model(generated)  # ← पहले outputs = model(generated)
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs  # ← ये लाइन जोड़ो (सुरक्षित)

        next_logits = logits[:, -1, :]

        # repetition penalty (तुम्हारा मूल)
        if repetition_penalty != 1.0:
            for i in range(generated.size(0)):
                unique_tokens = torch.unique(generated[i])
                for token_id in unique_tokens:
                    if next_logits[i, token_id] < 0:
                        next_logits[i, token_id] *= repetition_penalty
                    else:
                        next_logits[i, token_id] /= repetition_penalty

        # temperature
        next_logits = next_logits / temperature

        # NaN / inf check (ये जोड़ो – corruption रोकने के लिए)
        if torch.isnan(next_logits).any() or torch.isinf(next_logits).any():
            print("Warning: NaN or Inf in logits – stopping generation")
            break

        probs = torch.softmax(next_logits, dim=-1)

        # top-k
        if top_k > 0:
            top_k_vals, top_k_indices = torch.topk(probs, top_k)
            probs_zeroed = torch.zeros_like(probs).scatter_(-1, top_k_indices, top_k_vals)
            probs = probs_zeroed / probs_zeroed.sum(dim=-1, keepdim=True)

        # top-p (तुम्हारा मूल)
        sorted_probs, sorted_indices = torch.sort(probs, descending=True)
        cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = False
        sorted_probs = sorted_probs.masked_fill(sorted_indices_to_remove, 0.0)

        if sorted_probs.sum() == 0:
            next_token = torch.argmax(probs, dim=-1, keepdim=True)
        else:
            sorted_probs /= sorted_probs.sum(dim=-1, keepdim=True)
            next_token = torch.multinomial(sorted_probs, num_samples=1)
            next_token = sorted_indices.gather(-1, next_token)

        generated = torch.cat([generated, next_token], dim=1)

        if next_token.item() == sp.eos_id():
            break

    return generated

In [38]:
import re
import fitz          # PyMuPDF
import numpy as np
import torch
import faiss

# तुम्हारे पहले के clean फंक्शन (बिल्कुल वैसे ही)
def clean_hindi_text(text):
    if not text:
        return ""
    # Remove non-printable characters
    text = re.sub(r'[\x00-\x1F\x7F]', ' ', text)
    # Fix common PDF junk chars
    text = re.sub(r'[�•ﬁﬂ–—]', ' ', text)
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def fix_pdf_text(raw_text):
    text = re.sub(r'\s+', ' ', raw_text)
    text = re.sub(r'([ऀ-ॿ])([A-Za-z0-9])', r'\1 \2', text)
    text = re.sub(r'([a-zA-Z0-9])([ऀ-ॿ])', r'\1 \2', text)
    return text.strip()

# तुम्हारे दो chunking फंक्शन (दोनों रखे हैं, लेकिन sentence_chunk_text ज्यादा इस्तेमाल होगा)
def chunk_text(text, max_tokens=250, overlap=50):
    tokens = sp.encode(text)
    chunks = []
    i = 0
    while i < len(tokens):
        chunk = tokens[i:i + max_tokens]
        chunks.append(sp.decode(chunk))
        i += max_tokens - overlap
    return chunks

def sentence_chunk_text(text, max_chars=800, overlap_sentences=1):
    sentences = re.split(r'(?<=[।!?])\s+', text)
    chunks = []
    current_sents = []
    for sent in sentences:
        current_sents.append(sent)
        chunk_text = " ".join(current_sents)
        if len(chunk_text) >= max_chars:
            chunks.append(chunk_text.strip())
            current_sents = current_sents[-overlap_sentences:]
    if current_sents:
        chunks.append(" ".join(current_sents).strip())
    return chunks

# तुम्हारा index बनाने वाला हिस्सा
def build_index(chunks):
    texts = ["passage: " + chunk for chunk in chunks]
    embeddings = embed_model.encode(texts, normalize_embeddings=True, batch_size=32)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings.astype(np.float32))
    return index, chunks

# ------------------ अब मुख्य get_answer फंक्शन में सब integrate ------------------

def get_answer(query):
    # PDF लोड + क्लीन (हर बार query पर नहीं करना चाहिए, लेकिन अभी तुम्हारे स्टाइल में डाल रहे हैं)
    pdf_path = "story.pdf"
    doc = fitz.open(pdf_path)
    raw_text = ""
    for page in doc:
        raw_text += page.get_text("text")
        # raw_text += page.get_text("unicode")
    doc.close()
    
    fixed_text = fix_pdf_text(raw_text)
    cleaned_text = clean_hindi_text(fixed_text)
    
    # Chunking (तुम्हारा sentence level वाला इस्तेमाल कर रहे हैं)
    chunks = sentence_chunk_text(cleaned_text, max_chars=800, overlap_sentences=1)
    
    # Index बनाओ (हर बार बनाना inefficient है, लेकिन तुम्हारे मूल कोड के हिसाब से)
    global index  # अगर पहले से बना हो तो reuse, नहीं तो नया
    if 'index' not in globals() or index is None:
        index, chunks = build_index(chunks)   # chunks को update भी कर लेता है
    
    # Embed query (तुम्हारा तरीका)
    cleaned_query = query
    query_embedding = embed_model.encode([cleaned_query])
    
    # Search
    D, I = index.search(query_embedding, k=1)   # तुम्हारा मूल k=1
    
    retrieved_chunks = [chunks[i] for i in I[0]]
    
    # Print retrieved chunks (तुम्हारा मूल print)
    for idx, ch in zip(I[0], retrieved_chunks):
        print(f"\n--- Chunk {idx} ---\n{ch}\n")
    
    # context ids (तुम्हारा मूल तरीका)
    context_ids = []
    for chunk in retrieved_chunks:
        context_ids += sp.encode("[संदर्भ] " + chunk + "\n")
    
    # prompt ids (तुम्हारा मूल prompt बिल्कुल वैसा ही)
    prompt_ids = (
    sp.encode("""
सिर्फ नीचे दिए संदर्भ से जवाब दो। 
जवाब 2-4 वाक्यों से ज्यादा लंबा मत करो। 
एक ही बात बार-बार मत दोहराओ। 
अगर संदर्भ में स्पष्ट जवाब नहीं है तो सिर्फ लिखो: "संदर्भ में जानकारी नहीं है।"

[संदर्भ शुरू]
""")
    + context_ids
    + sp.encode(f"[संदर्भ खत्म]\nप्रश्न: {query}\nउत्तर:")
)


    print("\n=== Prompt IDs की लंबाई और कुछ शुरुआती tokens ===")
    print("Length:", len(prompt_ids))
    print("First 20 tokens:", prompt_ids[:20])
    print("Decoded without junk attempt:", sp.decode(prompt_ids[:100]).replace("⁇", ""))


    # 1. Retrieved chunks print करो (already तेरा print है, लेकिन बेहतर बना देते हैं)
    print("\n=== Retrieved Context (जो model को दिख रहा है) ===")
    for i, chunk in enumerate(retrieved_chunks, 1):
        print(f"Chunk {i} (index {I[0][i-1]}):")
        print(chunk.strip())
        print("-" * 80)

    # 2. Context ids से बना पूरा prompt text decode करके print करो
    full_prompt_text = sp.decode(prompt_ids)
    
    print("\n=== पूरा Prompt जो model को input जा रहा है ===")
    print(full_prompt_text)
    print("=" * 100)
    
    
    # input_ids torch tensor में
    input_ids = torch.tensor([prompt_ids]).to(DEVICE)   # DEVICE तुम्हारे कोड में define होना चाहिए (जैसे "cuda" या "cpu")
    
    # तुम्हारा generate फंक्शन कॉल (बिल्कुल वैसा ही)
    output_ids = generate(
        model,
        input_ids,
        max_new_tokens=120,           # ← बहुत कम करो, repetition जल्दी शुरू होता है
        temperature=0.7,              # 0.5 से थोड़ा ऊपर — ज्यादा deterministic
        top_p=0.90,
        repetition_penalty=1.20,      # 1.1 से थोड़ा बढ़ाओ लेकिन 1.5 मत जाना
        top_k=35                      # कम tokens → focus बेहतर
    )
    
    # decode सिर्फ नए tokens
    generated_ids = output_ids[0, input_ids.shape[1]:].tolist()
    answer = sp.decode(generated_ids)
    
    return answer


# इस्तेमाल का तरीका
query = "ट्रेन यात्रा और जीवन के बीच समानताएँ क्या हैं?"
answer = get_answer(query)
print("Generated answer:", answer)


--- Chunk 2 ---
क्योंकि जो कोशिश करता है, वह हारा हुआ नहीं होता। वह बस थोड़ा रुका हुआ होता है। और रुकना भी यात्रा का हिस्सा है। तो चलो, फिर से बैग उठाते हैं। फिर से टिकट चेक करते हैं। फिर से खिड़की के पास वाली सीट ढूँढते हैं। क्योंकि अभी बहुत सारे स्टेशन बाकी हैं। और हर स्टेशन पर कोई न कोई नई कहानी इंतज़ार कर रही है। धन्यवाद जीवन, तुम थोड़े जटिल हो, पर बहुत खूबसूरत भी हो। एक साधारण यात्री


=== Prompt IDs की लंबाई और कुछ शुरुआती tokens ===
Length: 193
First 20 tokens: [28843, 1, 28531, 1688, 863, 4694, 32, 1490, 182, 28869, 28843, 1, 26934, 2704, 28893, 28942, 28221, 32, 549, 6599]
Decoded without junk attempt:   सिर्फ नीचे दिए संदर्भ से जवाब दो।   जवाब 2-4 वाक्यों से ज्यादा लंबा मत करो।   एक ही बात बार-बार मत दोहराओ।   अगर संदर्भ में स्पष्ट जवाब नहीं है तो सिर्फ लिखो: "संदर्भ में जानकारी नहीं है।"  संदर्भ शुरू     संदर्भ   क्योंकि जो कोशिश करता है, वह हारा हुआ नहीं होता। वह बस थोड़ा रुका हुआ होता है। और रुकना भी यात्रा का हिस्सा है

=== Retrieved Context (जो model को दिख रहा है) ===


In [39]:
test_text = "ट्रेन यात्रा और जीवन के बीच समानताएँ क्या हैं? क्योंकि कोशिश करने वाला कभी हारा नहीं होता।"

tokens = sp.encode(test_text)
print("Tokens:", tokens)
print("Length:", len(tokens))
print("Decoded back:", sp.decode(tokens))
print("Pieces:", sp.encode_as_pieces(test_text))

Tokens: [1816, 1369, 44, 797, 11, 545, 8991, 2553, 496, 61, 28910, 796, 1221, 140, 965, 845, 24952, 100, 425, 28869]
Length: 20
Decoded back: ट्रेन यात्रा और जीवन के बीच समानताएँ क्या हैं? क्योंकि कोशिश करने वाला कभी हारा नहीं होता।
Pieces: ['▁ट्रेन', '▁यात्रा', '▁और', '▁जीवन', '▁के', '▁बीच', '▁समानता', 'एँ', '▁क्या', '▁हैं', '?', '▁क्योंकि', '▁कोशिश', '▁करने', '▁वाला', '▁कभी', '▁हारा', '▁नहीं', '▁होता', '।']


In [43]:
def get_answer(query):
    # PDF लोड + क्लीन + chunking + index + retrieval (वही रखा)
    pdf_path = "story.pdf"
    doc = fitz.open(pdf_path)
    raw_text = ""
    for page in doc:
        raw_text += page.get_text("text")
    doc.close()
    
    fixed_text = fix_pdf_text(raw_text)
    cleaned_text = clean_hindi_text(fixed_text)
    
    chunks = sentence_chunk_text(cleaned_text, max_chars=800, overlap_sentences=1)
    
    global index
    if 'index' not in globals() or index is None:
        index, chunks = build_index(chunks)
    
    cleaned_query = query
    query_embedding = embed_model.encode([cleaned_query])
    
    D, I = index.search(query_embedding, k=2)  # ← k=1 से बढ़ाकर 2 किया (बेहतर context)
    
    retrieved_chunks = [chunks[i] for i in I[0] if i < len(chunks)]
    
    # Retrieved chunks print
    print("\n=== Retrieved Context ===")
    for i, chunk in enumerate(retrieved_chunks, 1):
        print(f"Chunk {i} (index {I[0][i-1]}):")
        print(chunk.strip())
        print("-" * 80)
    
    # Prompt को STRING में पहले बनाओ (encode एक बार ही)
    context_str = "\n\n".join([f"[संदर्भ]\n{chunk.strip()}" for chunk in retrieved_chunks])
    
    full_prompt = f"""तुम सिर्फ नीचे दिए संदर्भ से जवाब दो।
जवाब 2-4 वाक्य से ज्यादा मत करो।
एक ही बात दोहराओ मत।
अगर साफ जानकारी नहीं है तो सिर्फ लिखो: "संदर्भ में जानकारी नहीं है।"

{context_str}

प्रश्न: {query}
उत्तर:"""
    
    # Encode एक बार
    prompt_ids = sp.encode(full_prompt)
    
    # Debug prints
    print("\n=== Prompt (string version) ===")
    print(full_prompt)
    print("\nTokens length:", len(prompt_ids))
    print("First 20 tokens:", prompt_ids[:20])
    print("=" * 100)
    
    input_ids = torch.tensor([prompt_ids], dtype=torch.long).to(DEVICE)
    
    # Generate (params improve)
    output_ids = generate(
        model,
        input_ids,
        max_new_tokens=90,          # बहुत कम रखो
        temperature=0.75,
        top_p=0.92,
        repetition_penalty=1.08,    # हल्का repetition control
        top_k=40
    )
    
    generated_ids = output_ids[0, input_ids.shape[1]:].tolist()
    raw_answer = sp.decode(generated_ids).strip()
    
    # Post-processing: repetition काटो + संक्षिप्त रखो
    sentences = re.split(r'[।?।!]\s*', raw_answer)
    clean_sentences = []
    seen = set()
    for sent in sentences:
        sent = sent.strip()
        if sent and sent not in seen and len(sent) > 5:
            seen.add(sent)
            clean_sentences.append(sent)
            if len(clean_sentences) >= 4:  # max 4 वाक्य
                break
    
    answer = "। ".join(clean_sentences).strip()
    if not answer:
        answer = raw_answer[:200] + "..." if len(raw_answer) > 200 else raw_answer
    
    # अगर repetition बहुत ज्यादा है तो fallback
    if answer.count("जीवन") > 4 or len(answer) > 350:
        answer = "संदर्भ के अनुसार: जीवन और ट्रेन यात्रा में रुकावटें आती हैं, लेकिन फिर से कोशिश करनी पड़ती है।"
    
    return answer

In [45]:
query = "ट्रेन यात्रा और जीवन के बीच समानताएँ क्या हैं?"
answer = get_answer(query)
print("Generated answer:", answer)


=== Retrieved Context ===
Chunk 1 (index 2):
क्योंकि जो कोशिश करता है, वह हारा हुआ नहीं होता। वह बस थोड़ा रुका हुआ होता है। और रुकना भी यात्रा का हिस्सा है। तो चलो, फिर से बैग उठाते हैं। फिर से टिकट चेक करते हैं। फिर से खिड़की के पास वाली सीट ढूँढते हैं। क्योंकि अभी बहुत सारे स्टेशन बाकी हैं। और हर स्टेशन पर कोई न कोई नई कहानी इंतज़ार कर रही है। धन्यवाद जीवन, तुम थोड़े जटिल हो, पर बहुत खूबसूरत भी हो। एक साधारण यात्री
--------------------------------------------------------------------------------
Chunk 2 (index 1):
खेतों में हल चलाते बैल, दूर पहाड़ों पर बादल का खेल, और बीच-बीच में उड़ते पंछी जो शायद किसी सपने की ओर जा रहे हैं। कभी-कभी सोचता हूँ सपने सच होते हैं या हम सपनों को सच मान लेते हैं? शायद दोनों। क्योंकि जो इंसान सपना देखना छोड़ देता है, वह जीना भी धीरे-धीरे छोड़ देता है। कल एक बच्चे ने मुझसे पूछा "अंकल, आप खुश हो?" मैंने हँसकर कहा "हाँ बेटा।" फिर मन ही मन सोचा खुशी कोई स्थायी अवस्था नहीं होती, वह एक छोटी-छोटी पलकों के झपकने के बीच आ-जा जाती है। जैसे सुबह की ओस, जैसे माँ की गो